<a href="https://colab.research.google.com/github/SupradeepDanturti/Unmask-the-Conversations/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%capture
!pip install speechbrain

In [8]:
import speechbrain as sb
import torch
import torchaudio
import os
import random
import librosa
import numpy as np
from scipy.io import wavfile

In [4]:
%%capture
# !gdown 1HYmz81UgfOBXC5QahfPtTq6T7NvTwTVt #trainclean100
# !gdown 1SJjSMlYmMtA1XU5fpTaVbU-9L6RzJXHM #openrirs

In [6]:
%%capture
from google.colab import drive
drive.mount('/content/drive/')

!tar -xvzf /content/drive/MyDrive/ConvAI/Project/rirs_noises.tar.gz
!tar -xvzf /content/drive/MyDrive/ConvAI/Project/train-clean-100.tar.gz

In [9]:
# Constants and Paths
LIBRISPEECH_PATH = "/content/LibriSpeech"
OPENRIR_PATH = "/content/rirs_noises"
NOISE_FOLDERS = [
    OPENRIR_PATH + "/pointsource_noises",
    OPENRIR_PATH + "real_rirs_isotropic_noises",
    OPENRIR_PATH + "simulated_rirs",
]
OUTPUT_PATH = "/content/noise_reverb_data"  # Destination for augmented data
AUGMENTATION_PROBABILITY = 0.5

In [17]:
import glob
import os
import torchaudio
from pathlib import Path
import pandas as pd

def parse_to_csv(libri_root_path, csv_filename="data.csv"):
    """
    Parses LibriSpeech-like data structured in a root directory (e.g., 'dev-clean-2')
    and creates a CSV file for use with speech processing libraries.

    Assumes files are named as 'spk_id-chapter_id-utterance_id.flac' with
    corresponding '.txt' transcription files in the same directory structure.

    Args:
        libri_root_path (str): The root directory of the LibriSpeech subset.
        csv_filename (str, optional): The name of the CSV file to create. Default is "data.csv".

    """

    flac_files = glob.glob(os.path.join(libri_root_path, "**/*.flac"), recursive=True)
    txt_files = glob.glob(os.path.join(libri_root_path, "**/*.txt"), recursive=True)

    # Dictionary to store parsed data
    data = {
        "ID": [],
        "file_path": [],
        "words": [],
        "spkID": [],
        "duration": []  # We'll calculate duration in seconds below
    }

    # Extract transcript information
    words_dict = {}
    for txt_file in txt_files:
        with open(txt_file, "r") as f:
            for line in f:
                line = line.strip("\n")
                utt_id, words = line.split(" ", 1)  # Split into utterance ID and words
                words_dict[utt_id] = words

    # Process FLAC files, get durations, and create CSV entries
    for flac_file in flac_files:
        utt_id = Path(flac_file).stem
        data['ID'].append(utt_id)
        data['file_path'].append(flac_file)
        data['words'].append(words_dict[utt_id])
        data['spkID'].append(utt_id.split("-")[0])

        audio_info = torchaudio.info(flac_file)
        duration = audio_info.num_frames / audio_info.sample_rate
        data['duration'].append(duration)

    # Create the CSV file
    df = pd.DataFrame(data)
    df.to_csv(csv_filename, index=False)

# Example usage
libri_root_path = "/content/LibriSpeech/train-clean-100"
parse_to_csv(libri_root_path)

In [18]:
df_librispeech = pd.read_csv("/content/data.csv")
df_librispeech.head()

,ID,file_path,words,spkID,duration
0,233-134440-0009,/content/LibriSpeech/train-clean-100/233/13444...,WASH WILLIAMS DID NOT ASSOCIATE WITH THE MEN O...,233,13.795
1,233-134440-0006,/content/LibriSpeech/train-clean-100/233/13444...,HIS NECK THIN HIS LEGS FEEBLE HE WAS DIRTY EVE...,233,15.795
2,233-134440-0032,/content/LibriSpeech/train-clean-100/233/13444...,SHE WAS A FOUL THING COME OUT A WOMAN MORE FOU...,233,13.465
3,233-134440-0039,/content/LibriSpeech/train-clean-100/233/13444...,HATRED HAD RAISED HIM TO THAT ELEVATION IT IS ...,233,14.905
4,233-134440-0011,/content/LibriSpeech/train-clean-100/233/13444...,WASH WILLIAMS WAS A MAN OF COURAGE A THING HAD...,233,15.715


In [19]:
from speechbrain.dataio.dataset import DynamicItemDataset

dataset = DynamicItemDataset.from_csv("data.csv")

In [26]:
print(dataset[list(dataset.keys())["233-134440-0009"]].keys())

AttributeError: 'DynamicItemDataset' object has no attribute 'keys'

In [13]:
import glob
import os
import pandas as pd

def parse_openrir_to_csv(rir_root_path, csv_filename="noise_data.csv"):
    """
    Parses the OpenRIR dataset for noise files and creates a CSV for convenient loading.

    Args:
        rir_root_path (str): Root directory of your OpenRIR dataset.
        csv_filename (str, optional):  Output CSV filename. Defaults to "noise_data.csv".
    """

    all_rooms = os.listdir(rir_root_path)
    data = {
        "id": [],
        "file_path": [],
        "room": [],
        "duration": []
    }

    for room in all_rooms:
        rir_files = glob.glob(os.path.join(rir_root_path, room, "*.wav"))

        for rir_file in rir_files:
            noise_id = os.path.basename(rir_file).replace(".wav", "")
            data["id"].append(noise_id)
            data["file_path"].append(rir_file)
            data["room"].append(room)

            #  Calculate duration (using your preferred audio library)
            audio_info = torchaudio.info(rir_file)
            duration = audio_info.num_frames / audio_info.sample_rate
            data["duration"].append(duration)

    df = pd.DataFrame(data)
    df.to_csv(csv_filename, index=False)

# Example usage
rir_root_path = "/content/rirs_noises"
parse_openrir_to_csv(rir_root_path)

RuntimeError: Failed to create input filter: "time_base=1/16000:sample_rate=16000:sample_fmt=s16:channel_layout=0x0" (Invalid argument)